In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession 

sc = SparkContext()
spark = SparkSession.builder.appName("HW4-1").getOrCreate()

In [53]:
spotify = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/training_set/log_0_20180715_000000000000.csv")

In [4]:
#feature = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/track_features")

## Merge Data

In [5]:
df = spotify.join(feature, spotify.track_id_clean == feature.track_id , "inner")
df.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- session_position: string (nullable = true)
 |-- session_length: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: string (nullable = true)
 |-- skip_2: string (nullable = true)
 |-- skip_3: string (nullable = true)
 |-- not_skipped: string (nullable = true)
 |-- context_switch: string (nullable = true)
 |-- no_pause_before_play: string (nullable = true)
 |-- short_pause_before_play: string (nullable = true)
 |-- long_pause_before_play: string (nullable = true)
 |-- hist_user_behavior_n_seekfwd: string (nullable = true)
 |-- hist_user_behavior_n_seekback: string (nullable = true)
 |-- hist_user_behavior_is_shuffle: string (nullable = true)
 |-- hour_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- premium: string (nullable = true)
 |-- context_type: string (nullable = true)
 |-- hist_user_behavior_reason_start: string (nullable = true)
 |-- hist_user_behavior_reason_end: string

In [6]:
df.count()

2990609

In [7]:
df.na.drop().count()

2990609

In [10]:
df.cache()

DataFrame[session_id: string, session_position: string, session_length: string, track_id_clean: string, skip_1: string, skip_2: string, skip_3: string, not_skipped: string, context_switch: string, no_pause_before_play: string, short_pause_before_play: string, long_pause_before_play: string, hist_user_behavior_n_seekfwd: string, hist_user_behavior_n_seekback: string, hist_user_behavior_is_shuffle: string, hour_of_day: string, date: string, premium: string, context_type: string, hist_user_behavior_reason_start: string, hist_user_behavior_reason_end: string, track_id: string, duration: string, release_year: string, us_popularity_estimate: string, acousticness: string, beat_strength: string, bounciness: string, danceability: string, dyn_range_mean: string, energy: string, flatness: string, instrumentalness: string, key: string, liveness: string, loudness: string, mechanism: string, mode: string, organism: string, speechiness: string, tempo: string, time_signature: string, valence: string, 

## Change Data Type

In [3]:
spotify.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- session_position: string (nullable = true)
 |-- session_length: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: string (nullable = true)
 |-- skip_2: string (nullable = true)
 |-- skip_3: string (nullable = true)
 |-- not_skipped: string (nullable = true)
 |-- context_switch: string (nullable = true)
 |-- no_pause_before_play: string (nullable = true)
 |-- short_pause_before_play: string (nullable = true)
 |-- long_pause_before_play: string (nullable = true)
 |-- hist_user_behavior_n_seekfwd: string (nullable = true)
 |-- hist_user_behavior_n_seekback: string (nullable = true)
 |-- hist_user_behavior_is_shuffle: string (nullable = true)
 |-- hour_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- premium: string (nullable = true)
 |-- context_type: string (nullable = true)
 |-- hist_user_behavior_reason_start: string (nullable = true)
 |-- hist_user_behavior_reason_end: string

In [54]:
columns_to_drop = ["hist_user_behavior_reason_start", "hist_user_behavior_reason_end", "context_type", "date"
                  ,"hour_of_day", "hist_user_behavior_is_shuffle", "hist_user_behavior_n_seekfwd", "session_length","session_position"]
df_drop = spotify.drop(*columns_to_drop)
df_drop.show(1)

+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|no_pause_before_play|short_pause_before_play|long_pause_before_play|hist_user_behavior_n_seekback|premium|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|             0|                   0|                      0|                     0|                            0|   true|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
only showi

In [55]:
from pyspark.sql.types import *

for c in ["skip_1", "skip_2", "skip_3", "not_skipped", "context_switch", "no_pause_before_play",
         "short_pause_before_play", "long_pause_before_play", "premium"]:
    df_drop = df_drop.withColumn(c, df_drop[c].cast(BooleanType()))
for c in ["hist_user_behavior_n_seekback"]:
    df_drop = df_drop.withColumn(c, df_drop[c].cast(IntegerType()))  
    
df_drop.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: boolean (nullable = true)
 |-- skip_2: boolean (nullable = true)
 |-- skip_3: boolean (nullable = true)
 |-- not_skipped: boolean (nullable = true)
 |-- context_switch: boolean (nullable = true)
 |-- no_pause_before_play: boolean (nullable = true)
 |-- short_pause_before_play: boolean (nullable = true)
 |-- long_pause_before_play: boolean (nullable = true)
 |-- hist_user_behavior_n_seekback: integer (nullable = true)
 |-- premium: boolean (nullable = true)



## Rating

In [25]:
df_drop.show(1)

+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|no_pause_before_play|short_pause_before_play|long_pause_before_play|hist_user_behavior_n_seekback|premium|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|         false|               false|                  false|                 false|                            0|   true|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
only showi

In [63]:
def pause_udf(short_pause, long_pause):
    if short_pause or long_pause:
        return 0
    else:
        return 1
    
pause = spark.udf.register("pause", pause_udf)

df1 = df_drop.select("session_id", pause(col("short_pause_before_play"), col("long_pause_before_play")).alias("Not_Pause"))
df1.show(5)

df_drop1 = df_drop.join(df1, ["session_id"] , "left")
df_drop1 = df_drop1.withColumn("Not_Pause", df_drop1["Not_Pause"].cast(IntegerType()))

+--------------------+---------+
|          session_id|Not_Pause|
+--------------------+---------+
|0_00006f66-33e5-4...|        1|
|0_00006f66-33e5-4...|        1|
|0_00006f66-33e5-4...|        1|
|0_00006f66-33e5-4...|        1|
|0_00006f66-33e5-4...|        1|
+--------------------+---------+
only showing top 5 rows



In [64]:
def skip_rating_udf(premium, skip_1, skip_2, skip_3, not_skip, nopause):
    if premium:
        if skip_1:
            skip1 = 1 - (1 - nopause)
        else:
            skip1 = 0
        if skip_2:
            skip2 = 3
        else:
            skip2 = 0
        if skip_3:
            skip3 = 5
        else:
            skip3 = 0
        if not_skip:
            notskip = 6 + (2 - 2 * nopause)
        else:
            notskip = 0
        return max(skip1, skip2, skip3, notskip)
    else:
        if skip_1:
            skip1 = 0
        else:
            skip1 = 0
        if skip_2:
            skip2 = 1
        else:
            skip2 = 0
        if skip_3:
            skip3 = 2
        else:
            skip3 = 0
        if not_skip:
            notskip = 5 + (1 - 1 * nopause)
        else:
            notskip = 0
        return max(skip1, skip2, skip3, notskip)
    
skip_rating = spark.udf.register("skip_rating", skip_rating_udf)

df2 = df_drop1.select(col("session_id"), skip_rating(col("premium"), col("skip_1"), 
                                               col("skip_2"), col("skip_3"), col("not_skipped"), 
                                               col("Not_Pause")).alias("Skip_Rating"))
df2.show(5)

df_drop2 = df_drop1.join(df2, ["session_id"] , "left")
df_drop2 = df_drop2.withColumn("Skip_Rating", df_drop2["Skip_Rating"].cast(IntegerType()))

+--------------------+-----------+
|          session_id|Skip_Rating|
+--------------------+-----------+
|0_000cec57-79f9-4...|          2|
|0_000cec57-79f9-4...|          2|
|0_000cec57-79f9-4...|          2|
|0_000cec57-79f9-4...|          2|
|0_000cec57-79f9-4...|          2|
+--------------------+-----------+
only showing top 5 rows



In [68]:
def final_rating_udf(Skip_Rating, context_switch, seekback):
    if context_switch:
        context = 1
    else:
        context = 0
    inside = 1 - (Skip_Rating + 2*context)/10
    return (1 - pow(inside, (pow(seekback, 1/3)))) * 10

final_rating = spark.udf.register("final_rating", final_rating_udf)

df3 = df_drop2.select("session_id", final_rating(col("Skip_Rating"), col("context_switch"),
                                        col("hist_user_behavior_n_seekback")).alias("Final_Rating"))
df3.show(5)

df_drop3 = df_drop2.join(df3, ["session_id"] , "left")
df_drop3 = df_drop3.withColumn("Final_Rating", df_drop3["Final_Rating"].cast(IntegerType()))

+--------------------+------------+
|          session_id|Final_Rating|
+--------------------+------------+
|0_000cec57-79f9-4...|         0.0|
|0_000cec57-79f9-4...|         0.0|
|0_000cec57-79f9-4...|         0.0|
|0_000cec57-79f9-4...|         0.0|
|0_000cec57-79f9-4...|         0.0|
+--------------------+------------+
only showing top 5 rows



In [69]:
df_drop3.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: boolean (nullable = true)
 |-- skip_2: boolean (nullable = true)
 |-- skip_3: boolean (nullable = true)
 |-- not_skipped: boolean (nullable = true)
 |-- context_switch: boolean (nullable = true)
 |-- no_pause_before_play: boolean (nullable = true)
 |-- short_pause_before_play: boolean (nullable = true)
 |-- long_pause_before_play: boolean (nullable = true)
 |-- hist_user_behavior_n_seekback: integer (nullable = true)
 |-- premium: boolean (nullable = true)
 |-- Not_Pause: integer (nullable = true)
 |-- Skip_Rating: integer (nullable = true)
 |-- Final_Rating: integer (nullable = true)



In [75]:
df_final = df_drop3.select("session_id", "track_id_clean", "Final_Rating")

In [ ]:
df_final.write.format("csv").option("header","true").mode("Overwrite").save("s3://olihw4/spotify_small_version")

## Distribution 

In [ ]:
df_final.select("Final_Rating").rdd.flatMap(lambda x: x).histogram(10)

In [ ]:
def describe_pd(df_in, columns, deciles=False):
    '''
    Function to union the basic stats results and deciles
    :param df_in: the input dataframe
    :param columns: the cloumn name list of the numerical variable
    :param deciles: the deciles output

    :return : the numerical describe info. of the input dataframe

    :author: Ming Chen and Wenqiang Feng
    :email:  von198@gmail.com
    '''

    if deciles:
        percentiles = np.array(range(0, 110, 10))
    else:
        percentiles = [25, 50, 75]

    percs = np.transpose([np.percentile(df_in.select(x).collect(), percentiles) for x in columns])
    percs = pd.DataFrame(percs, columns=columns)
    percs['summary'] = [str(p) + '%' for p in percentiles]

    spark_describe = df_in.describe().toPandas()
    new_df = pd.concat([spark_describe, percs],ignore_index=True)
    new_df = new_df.round(2)
    return new_df[['summary'] + columns]

In [ ]:
describe_pd(df_final, "Final_Rating" ,deciles=True)